# Hobohm 1

### Fill in the blanked out part of the code (XX)

## Python Imports

In [137]:
import numpy as np
from time import time

## Data Imports

## DEFINE THE PATH TO YOUR COURSE DIRECTORY

In [138]:
data_dir = "C:/Users/vinde/OneDrive/Dokumenter/DTU/Algorithms_in_bioinformatics/Project/Cluster_data/"
data_dir_2 = "C:/Users/vinde/OneDrive/Dokumenter/DTU/Algorithms_in_bioinformatics/data/"

In [139]:
alphabet_file = data_dir_2 + "Matrices/alphabet"
alphabet = np.loadtxt(alphabet_file, dtype=str)

alphabet

blosum_file = data_dir_2 + "Matrices/BLOSUM50"
_blosum50 = np.loadtxt(blosum_file, dtype=int).T

blosum50 = {}

for i, letter_1 in enumerate(alphabet):
    
    blosum50[letter_1] = {}

    for j, letter_2 in enumerate(alphabet):
        
        blosum50[letter_1][letter_2] = _blosum50[i, j]
        
#blosum50      

### Sequences

In [144]:
def load_sequences():
    
    #database_file = data_dir_2 + "Hobohm/database_list.tab"
    #database_list = np.loadtxt(database_file, dtype=str).reshape(-1,2)

    database_file = data_dir + "A0201.txt"
    database_list = np.loadtxt(database_file, dtype=str).reshape(-1,2)
    
    #ids = database_list[:, 0]
    #sequences = database_list[:, 1]

    ids = database_list[:, 0]
    sequences = database_list[:, 0]

    return sequences, ids

## Smith-Waterman O2

### This code is identical to the code you wrote the other day

In [145]:
def smith_waterman(query, database, scoring_scheme, gap_open, gap_extension):
    
    P_matrix, Q_matrix, D_matrix, E_matrix, i_max, j_max, max_score = smith_waterman_alignment(query, database, scoring_scheme, gap_open, gap_extension)
    
    aligned_query, aligned_database, matches = smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query, database, gap_open, gap_extension)
    
    return aligned_query, aligned_database, matches


def smith_waterman_alignment(query, database, scoring_scheme, gap_open, gap_extension):

    # Matrix imensions
    M = len(query)
    N = len(database)
    
    # D matrix change to float
    D_matrix = np.zeros((M+1, N+1), np.int)

    # P matrix
    P_matrix = np.zeros((M+1, N+1), np.int)
    
    # Q matrix
    Q_matrix = np.zeros((M+1, N+1), np.int)

    # E matrix
    E_matrix = np.zeros((M+1, N+1), dtype=object)

    # Main loop
    D_matrix_max_score, D_matrix_i_max, D_matrix_i_max = -9, -9, -9
    for i in range(M-1, -1, -1):
        for j in range(N-1, -1, -1):
            
            # Q_matrix[i,j] entry
            gap_open_database = D_matrix[i+1,j] + gap_open
            gap_extension_database = Q_matrix[i+1,j] + gap_extension
            max_gap_database = max(gap_open_database, gap_extension_database)
            
            Q_matrix[i,j] = max_gap_database
                
            # P_matrix[i,j] entry
            gap_open_query = D_matrix[i,j+1] + gap_open
            gap_extension_query = P_matrix[i,j+1] + gap_extension
            max_gap_query = max(gap_open_query, gap_extension_query)
            
            P_matrix[i,j] = max_gap_query
            
            # D_matrix[i,j] entry
            diagonal_score = D_matrix[i+1,j+1] + scoring_scheme[query[i]][database[j]]    
            
            # E_matrix[i,j] entry
            candidates = [(1, diagonal_score),
                          (2, gap_open_database),
                          (4, gap_open_query),
                          (3, gap_extension_database),
                          (5, gap_extension_query)]
            
            direction, max_score = max(candidates, key=lambda x: x[1])
            
            
            # check entry sign
            if max_score > 0:
                E_matrix[i,j] = direction
            else:
                E_matrix[i,j] = 0
            
            # check max score sign
            if max_score > 0:
                D_matrix[i, j] = max_score
            else:
                D_matrix[i, j] = 0

            # fetch global max score
            if max_score > D_matrix_max_score:
                D_matrix_max_score = max_score
                D_matrix_i_max = i
                D_matrix_j_max = j
            
    return P_matrix, Q_matrix, D_matrix, E_matrix, D_matrix_i_max, D_matrix_j_max, D_matrix_max_score


def smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query, database, gap_open, gap_extension):
    
    # Matrix imensions
    M = len(query)
    N = len(database)
    
    # aligned query string
    aligned_query = []
    
    # aligned database string
    aligned_database = []
    
    # total identical matches
    matches = 0

        
    # start from max_i, max_j
    i, j = i_max, j_max
    while i < M and j < N:

        # E[i,j] = 0, stop back tracking
        if E_matrix[i, j] == 0:
            break
        
        # E[i,j] = 1, match
        if E_matrix[i, j] == 1:
            aligned_query.append(query[i])
            aligned_database.append(database[j])
            if ( query[i] == database[j]):
                matches += 1
            i += 1
            j += 1
        
        
        # E[i,j] = 2, gap opening in database
        if E_matrix[i, j] == 2:
            aligned_database.append("-")
            aligned_query.append(query[i])
            i += 1

            
        # E[i,j] = 3, gap extension in database
        if E_matrix[i, j] == 3:
                   
            count = i + 2
            score = D_matrix[count, j] + gap_open + gap_extension
            
            # Find length of gap
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.00001):   
                count += 1
                score = D_matrix[count, j] + gap_open + (count-i-1)*gap_extension

            for k in range(i, count):
                aligned_database.append("-")
                aligned_query.append(query[i])
                i += 1
            
            
        # E[i,j] = 4, gap opening in query
        if E_matrix[i, j] == 4:
            aligned_query.append("-")
            aligned_database.append(database[j])
            j += 1
        
        
        # E[i,j] = 5, gap extension in query
        if E_matrix[i, j] == 5:
             
            count = j + 2
            score = D_matrix[i, count] + gap_open + gap_extension
            
            # Find length of gap
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.0001): 
                count += 1
                score = D_matrix[i, count] + gap_open + (count-j-1)*gap_extension

            for k in range(j, count):
                aligned_query.append("-")
                aligned_database.append(database[j])
                j += 1

                
    return aligned_query, aligned_database, matches

## Hobohm 1

### Similarity Function

### This code defines the threshold for similarity

In [146]:
def homology_function(alignment_length, matches):
    
    homology_score = 7
    #homology_score = XX, this was just Ole's formula
    #homology_score = 2.9*np.sqrt(alignment_length)**0.5
    
    #if matches XX homology_score: ## Add the inequally sign
    if matches > homology_score:
        return "discard", homology_score
    else:
        return "keep", homology_score

### Main Loop Initiation

In [147]:
# load list
candidate_sequences, candidate_ids = load_sequences()
print ("# Numner of elements:", len(candidate_sequences))

accepted_sequences, accepted_ids = [], []

accepted_sequences.append(candidate_sequences[0])
accepted_ids.append([candidate_ids[0]])
#print(accepted_ids)
print("# Unique.", 0, len(accepted_sequences)-1, accepted_ids[0][0])



# Numner of elements: 3089
# Unique. 0 0 YVMTMILFL


### Main Loop 

In [ ]:
# parameters
scoring_scheme = blosum50
gap_open = -11
gap_extension = -1

t0 = time()

for i in range(1, len(candidate_sequences)):
#for i in range(1, 41):

    for j in range(0, len(accepted_sequences)):

        query = candidate_sequences[i]
        database = accepted_sequences[j] 
        
        aligned_query, aligned_database, matches = smith_waterman(query, database, scoring_scheme, gap_open, gap_extension)
        
        alignment_length = len(aligned_query)
        
        homology_outcome, homology_score = homology_function(alignment_length, matches)
     
        # query is not unique
        if homology_outcome == "discard":
            
            accepted_ids[j].append(candidate_ids[i])
            print ("# Not unique.", i, candidate_ids[i], "is homolog to", accepted_ids[j][0], homology_score)
            
            break
            
            
    # query is unique
    if homology_outcome == "keep":
        #accepted_sequences.append(XX)
        accepted_sequences.append(candidate_sequences[i])
        #accepted_ids.append(XX)
        accepted_ids.append([candidate_ids[i]])

        print ("# Unique.", i, len(accepted_sequences)-1, candidate_ids[i], homology_score)

t1 = time()

print("Elapsed time (m):", (t1-t0)/60)

print ("Accepted clusters:", len(accepted_ids))
for i in range(len(accepted_ids)):
    print( accepted_ids[i])

# Unique. 1 1 YTYSGLFCV 7
# Unique. 2 2 YQSFLFWFL 7
# Unique. 3 3 YMMGIEYGL 7
# Unique. 4 4 YLYALYSPL 7
# Unique. 5 5 YLSSWTPVV 7
# Unique. 6 6 YLSAKVYML 7
# Unique. 7 7 YLMPYSVYI 7
# Unique. 8 8 YLMDKLNLT 7
# Unique. 9 9 YLLFNHFSV 7
# Unique. 10 10 YLFGGFSTL 7
# Unique. 11 11 YLDFLLLLL 7
# Unique. 12 12 YITALNHLV 7
# Unique. 13 13 YILCNMALL 7
# Unique. 14 14 YIIDWMVDI 7
# Unique. 15 15 WLYGYNFII 7
# Unique. 16 16 WLIGFDFDV 7
# Unique. 17 17 VMYAFTTPL 7
# Unique. 18 18 VLLTYFCFV 7
# Unique. 19 19 VLDTTLYAV 7
# Unique. 20 20 VLALYSPPL 7
# Unique. 21 21 TLYDFDYYI 7
# Unique. 22 22 TLLGLILFV 7
# Unique. 23 23 TLIDIWFLA 7
# Unique. 24 24 TLAPFNFLV 7
# Unique. 25 25 SMSSYDFST 7
# Unique. 26 26 SLQEEIAFL 7
# Unique. 27 27 SLLYLILFL 7
# Unique. 28 28 SLDVINYLI 7
# Unique. 29 29 SLAGGIIGV 7
# Unique. 30 30 SIFGFQAEV 7
# Unique. 31 31 RTFHIFYYL 7
# Unique. 32 32 RQIFIHYSV 7
# Unique. 33 33 RLVDFFPDI 7
# Unique. 34 34 RLFSYNFTT 7
# Unique. 35 35 RLAVYIDKV 7
# Unique. 36 36 RILPYTFKI 7
# Unique. 

# Unique. 271 257 GLLGWSPQA 7
# Unique. 272 258 KLLWFLTGT 7
# Unique. 273 259 FLLTRILTI 7
# Unique. 274 260 YIYYFFIRL 7
# Unique. 275 261 TIWAANAGV 7
# Unique. 276 262 FLLSLGIHL 7
# Unique. 277 263 YLYPGPVTV 7
# Unique. 278 264 MMAKEEELV 7
# Not unique. 279 LLTYFCFVA is homolog to VLLTYFCFV 7
# Unique. 280 265 HMYFTFFDV 7
# Unique. 281 266 YQYVRLHEM 7
# Unique. 282 267 YLYVDKNFI 7
# Unique. 283 268 YLVAYKATV 7
# Unique. 284 269 VLYPVIFIT 7
# Not unique. 285 YLAPGPVTA is homolog to YLWPGPVTA 7
# Unique. 286 270 SLLTFMIAA 7
# Unique. 287 271 YLKAYQATV 7
# Unique. 288 272 FVDTMSIYI 7
# Unique. 289 273 WLMKNMDPL 7
# Unique. 290 274 KMVELVHFL 7
# Unique. 291 275 SLSLHPLYV 7
# Unique. 292 276 KIMSIGFEA 7
# Unique. 293 277 GMFGGCFAA 7
# Unique. 294 278 SLIYYQNEV 7
# Unique. 295 279 GQWDGWVWL 7
# Unique. 296 280 KIAPGIADI 7
# Unique. 297 281 NIFMTLVPV 7
# Unique. 298 282 KLTPLCVTL 7
# Unique. 299 283 KTWGQYWQV 7
# Unique. 300 284 QIFEVYWYL 7
# Unique. 301 285 RLNDFLGLL 7
# Not unique. 302 ILAQ

# Unique. 527 491 LAALFMYYA 7
# Unique. 528 492 FICWHSSEI 7
# Unique. 529 493 KLAEIFQPF 7
# Unique. 530 494 ITNGYLISI 7
# Unique. 531 495 HLINKLLST 7
# Unique. 532 496 STISWMMKL 7
# Unique. 533 497 KVLEYVIKV 7
# Unique. 534 498 RMYSPISIL 7
# Unique. 535 499 TLFLLFLEI 7
# Not unique. 536 SLFNTVATL is homolog to SLFNTVATV 7
# Not unique. 537 ATLNTLITL is homolog to TLNTLITLI 7
# Unique. 538 500 VLQQSTYQL 7
# Unique. 539 501 RLIDFLKDV 7
# Unique. 540 502 GLMWLSYFV 7
# Unique. 541 503 HLREYQDLL 7
# Unique. 542 504 TLGTVILLV 7
# Unique. 543 505 GLVDLFVFS 7
# Unique. 544 506 FANSKFTLV 7
# Unique. 545 507 GLQDCTMLV 7
# Unique. 546 508 TLVDLCFWS 7
# Not unique. 547 FTDQVPFSV is homolog to FLDQVPFSV 7
# Unique. 548 509 RIFYNILEI 7
# Unique. 549 510 ILPDKIDGL 7
# Unique. 550 511 ILLEPVHGV 7
# Unique. 551 512 GLSRYVARL 7
# Not unique. 552 IMIGVLVGV is homolog to IMMGVLVGV 7
# Unique. 553 513 STYPGNTFV 7
# Not unique. 554 SLYAVSPSV is homolog to SLYADSPSV 7
# Unique. 555 514 FKNSSNLFT 7
# Unique. 

In [136]:
print("Elapsed time (m):", (t1-t0)/60)

print ("Accepted clusters:", len(accepted_ids))
for i in range(len(accepted_ids)):
    print( accepted_ids[i])

Elapsed time (m): -31.348621559143066
Accepted clusters: 1
['YQSFLFWFL;1.0']
